# Data pre-processing

Description: The script will pre-process the data. This step of pre-processing will include: removing rows with missing values, binary encoding categorical data

Author: Caroline Risoud

License:  CC-BY-4.0

Last update date: 23.10.2021

In [1]:
import pandas as pd

from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import log_loss

import xgboost as xgb

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import matplotlib.pyplot as plt
import matplotlib

In [2]:
# Reading the csv data file
df_train_validate = pd.read_csv('nhts_train_validate.csv', index_col='TRIPID')

### Missing values in the dataset

In the following cell:

- We create a dataframe 'df_null_values' which will list all the columns (features, labels) of the dataset and will show how many NULL (None, NaN) values each of the columns counts.

- We see that NULL values (exactly 8) are only found in the TRAVELMODE column which is the target/label column.


In [3]:
# Finding the amount of NULL values

# For each column, total stores the number of null values
total = df_train_validate.isnull().sum().sort_values(ascending=False)


# For each column, percent translater the number of null values into a percentage
percent = total/len(df_train_validate)*100


# Dataframe with Total as the first column and Percent as the second one
df_null_values = pd.concat([total,percent], axis=1, keys=['Total', 'Percent'])

- We will drop the 8 rows with missing TRAVELMODE. We assume that these row removals will not introduce sampling bias because. 

In [4]:
df_train_validate = df_train_validate.dropna(how='any', axis=0)

In [5]:
df_train_validate

,HOUSEID,PERSONID,TDTRPNUM,STRTTIME,TRPMILES,TRPTRANS,LOOP_TRIP,TRIPPURP,TRAVDAY,HOMEOWN,...,EDUC,WORKER,WHYTRP90,R_AGE_IMP,R_SEX_IMP,OBHUR,DBHUR,OBPPOPDN,DBPPOPDN,TRAVELMODE
TRIPID,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,615,2.140,1,2,HBSHOP,3,2,...,3,1,10,29,2,U,U,7000,7000,WALK
2,1,1,2,703,2.426,11,2,NHB,3,2,...,3,1,4,29,2,U,S,7000,7000,RAIL
3,1,1,3,735,2.752,11,2,HBO,3,2,...,3,1,10,29,2,S,U,7000,7000,RAIL
4,1,1,4,1500,2.752,11,2,HBO,3,2,...,3,1,4,29,2,U,S,7000,7000,RAIL
5,1,1,5,1612,1.057,1,2,NHB,3,2,...,3,1,3,29,2,S,U,7000,7000,WALK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16137,3303,1,3,1300,1.876,3,2,NHB,6,1,...,3,2,3,92,2,C,C,7000,3000,DRIVE
16138,3303,1,4,1325,0.239,3,2,NHB,6,1,...,3,2,3,92,2,C,C,3000,3000,DRIVE
16139,3303,1,5,1440,3.683,3,2,HBSHOP,6,1,...,3,2,3,92,2,C,S,3000,3000,DRIVE


### Categorical data - Binary encoding

All data must be numerical.

For our target label, TRAVELMODE, we choose numerical encoding. Indeed, we want to have one column for the target categorical value which leads to the use of linear encoding to achieve that.

For the features, we choose One-hot encoding (Binary encoding) as a solution. On one hand, this solution does not impute an order and distance compared to a numerical encoding. On the other hand, it makes the data much wider by creating many more features.

In [6]:
# shows us the type of each column - the object types are the ones to be encoded

display(df_train_validate.dtypes)

HOUSEID         int64
PERSONID        int64
TDTRPNUM        int64
STRTTIME        int64
TRPMILES      float64
TRPTRANS        int64
LOOP_TRIP       int64
TRIPPURP       object
TRAVDAY         int64
HOMEOWN         int64
HHSIZE          int64
HHFAMINC        int64
HHSTATE        object
WRKCOUNT        int64
LIF_CYC         int64
URBAN           int64
URBANSIZE       int64
CENSUS_D        int64
HH_RACE         int64
EDUC            int64
WORKER          int64
WHYTRP90        int64
R_AGE_IMP       int64
R_SEX_IMP       int64
OBHUR          object
DBHUR          object
OBPPOPDN        int64
DBPPOPDN        int64
TRAVELMODE     object
dtype: object

In [7]:
# numerical encoding for the target categorical lable TRAVELMODE

str_to_val = {
    'WALK': 0,
    'CYCLE': 1,
    'RAIL': 2,
    'BUS': 3,
    'DRIVE': 4,
    'PASSENGER': 5,
    'TAXI': 6,
    'OTHER': 7
}

# Replacing the strings with their respective values
df_train_validate.TRAVELMODE.replace(str_to_val, inplace=True)

df_train_validate.head()

,HOUSEID,PERSONID,TDTRPNUM,STRTTIME,TRPMILES,TRPTRANS,LOOP_TRIP,TRIPPURP,TRAVDAY,HOMEOWN,...,EDUC,WORKER,WHYTRP90,R_AGE_IMP,R_SEX_IMP,OBHUR,DBHUR,OBPPOPDN,DBPPOPDN,TRAVELMODE
TRIPID,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,615,2.140,1,2,HBSHOP,3,2,...,3,1,10,29,2,U,U,7000,7000,0
2,1,1,2,703,2.426,11,2,NHB,3,2,...,3,1,4,29,2,U,S,7000,7000,2
3,1,1,3,735,2.752,11,2,HBO,3,2,...,3,1,10,29,2,S,U,7000,7000,2
4,1,1,4,1500,2.752,11,2,HBO,3,2,...,3,1,4,29,2,U,S,7000,7000,2
5,1,1,5,1612,1.057,1,2,NHB,3,2,...,3,1,3,29,2,S,U,7000,7000,0


In [8]:
# Binary encoding for the 4 remaining categorical features: 'TRIPPURP','HHSTATE', 'OBHUR', 'DBHUR'

categorical_cols = [
    'TRIPPURP',
    'HHSTATE',
    'OBHUR',
    'DBHUR'
]


df_processed = pd.get_dummies(
    df_train_validate, prefix_sep=':', columns=categorical_cols)

df_processed.head()

,HOUSEID,PERSONID,TDTRPNUM,STRTTIME,TRPMILES,TRPTRANS,LOOP_TRIP,TRAVDAY,HOMEOWN,HHSIZE,...,OBHUR:R,OBHUR:S,OBHUR:T,OBHUR:U,DBHUR:-9,DBHUR:C,DBHUR:R,DBHUR:S,DBHUR:T,DBHUR:U
TRIPID,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,615,2.140,1,2,3,2,3,...,0,0,0,1,0,0,0,0,0,1
2,1,1,2,703,2.426,11,2,3,2,3,...,0,0,0,1,0,0,0,1,0,0
3,1,1,3,735,2.752,11,2,3,2,3,...,0,1,0,0,0,0,0,0,0,1
4,1,1,4,1500,2.752,11,2,3,2,3,...,0,0,0,1,0,0,0,1,0,0
5,1,1,5,1612,1.057,1,2,3,2,3,...,0,1,0,0,0,0,0,0,0,1


### Internal and external validation

 The test data here can not be used for external validation because it DOES NOT include the choice label.
 
In order to create a test set that represents external validation, we will seperate the train_validate set into train_validation and test set. Cross-validation to optimize the hyperparameters will be carried out on the train_validate set and finally the test set can be used for external validation. 
 
    train_validate data:
        --> 80 % train_validate_rev
        --> 20 % test_rev
 

The Cross-validation with random search will be used to optimise the hyperparameters, it will also account for the hierarchical nature of the data. It will be used as follows:

 - Train on 4 folds, test on 1 fold
 - Training data: 80% of the train_validate_rev
 - Test data: 20% of the train_validate_rev
 
 Random sampling of validation folds
 --> INTERNAL VALIDATION
 
     
 The Test:
 
 - Training data: 100% of the train_validate_rev data with the optimal hyperparameters found previously
 - Test data: 100% of the test_rev
 
 --> EXTERNAL VALIDATION

Splitting train_validate dataframe by row index into train and validate dataframe:

- 80% of the df_train_validate is assigned to df_train_validate_rev
- 20% of the df_train_validate is assigned to df_test_rev

In [9]:
# defining index 1
id1 = round(len(df_processed)*0.8)

df_train_validate_rev = df_processed.iloc[:id1,:]
df_test_rev = df_processed.iloc[id1:,:]

print("Shape of new dataframes - {} , {}".format(df_train_validate_rev.shape, df_test_rev.shape))


Shape of new dataframes - (12906, 91) , (3227, 91)


In [10]:
# We extract the features and labels, removing the id and context columns because they can't be inputed in the ML model

target = ['TRAVELMODE']

id_context = ['TRIPID', 
              'HOUSEID', 
              'PERSONID', 
              'TDTRPNUM',
              'LOOP_TRIP'
             ]



features = [c for c in df_processed.columns 
            if c not in (target + id_context)]

# y is the target label (here: 'TRAVELMODE')
# X are the features that are inputed to the model to predict the target label

# ravel() is used to flatten the multi-dimensional array to a vector
y = df_processed[target].values.ravel()
X = df_processed[features]

y_train_validate_rev = df_train_validate_rev[target].values.ravel()
X_train_validate_rev = df_train_validate_rev[features]

y_test_rev = df_test_rev[target].values.ravel()
X_test_rev = df_test_rev[features]

In [22]:
# features in the train and validate set

X_train_validate_rev

,STRTTIME,TRPMILES,TRPTRANS,TRAVDAY,HOMEOWN,HHSIZE,HHFAMINC,WRKCOUNT,LIF_CYC,URBAN,...,OBHUR:R,OBHUR:S,OBHUR:T,OBHUR:U,DBHUR:-9,DBHUR:C,DBHUR:R,DBHUR:S,DBHUR:T,DBHUR:U
TRIPID,,,,,,,,,,,,,,,,,,,,,
1,615,2.140,1,3,2,3,5,1,3,1,...,0,0,0,1,0,0,0,0,0,1
2,703,2.426,11,3,2,3,5,1,3,1,...,0,0,0,1,0,0,0,1,0,0
3,735,2.752,11,3,2,3,5,1,3,1,...,0,1,0,0,0,0,0,0,0,1
4,1500,2.752,11,3,2,3,5,1,3,1,...,0,0,0,1,0,0,0,1,0,0
5,1612,1.057,1,3,2,3,5,1,3,1,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12910,745,1.104,10,3,2,5,1,1,4,1,...,0,0,0,1,0,0,0,0,0,1
12911,1420,1.118,10,3,2,5,1,1,4,1,...,0,0,0,1,0,0,0,0,0,1
12912,1045,1.103,10,3,2,5,1,1,4,1,...,0,0,0,1,0,0,0,0,0,1


In [13]:
# storing the variables to re-use them in the Hyperparmaters.ipynb file

%store X_train_validate_rev
%store y_train_validate_rev
%store X_test_rev
%store y_test_rev

Stored 'X_train_validate_rev' (DataFrame)
Stored 'y_train_validate_rev' (ndarray)
Stored 'X_test_rev' (DataFrame)
Stored 'y_test_rev' (ndarray)
